In [ ]:
import sys
import os
import random
import datetime
import re
import math
import logging
from collections import OrderedDict
import multiprocessing
import numpy as np
import tensorflow as tf
import keras
import keras.backend as K
import keras.layers as KL
import keras.engine as KE
import keras.models as KM

import index_tracker as track   
import csv
from keras.callbacks import LearningRateScheduler
import cv2
import scipy
import h5py
import skimage
import time

# set seed
from numpy.random import seed
seed(1234)
from tensorflow import set_random_seed
set_random_seed(1234)

from semseg import utils, resnet, detection, losses, data_generator as dg, visualize, metrics
from semseg.config import Config

from semseg import semseg_model as modellib
import dicom_to_numpy as dtn
import matplotlib.pyplot as plt

from semseg import testing_semseg

**1. Set training hyperparameters**

In [ ]:
class TumorConfig(Config):    
    NAME = 'lung_tumor'
    LOSS_WEIGHTS = {           
        "rpn_mask_loss": 1.        
    }
    LEARNING_RATE = 0.001
    IMAGES_PER_GPU = 16
    TRAIN_BN = None  
    IMAGE_HEIGHT = 384
    IMAGE_WIDTH = 384
    IMAGE_SHAPE = [384, 384, 1]
    STEPS_PER_EPOCH = 1000
    VALIDATION_STEPS = 181   
    
    IMAGE_CHANNEL_COUNT = 1
   

# SET UP CONFIG
config = TumorConfig()

**2. Load training and validation datasets**

In [ ]:
data_path = ''
patient_name = 'patient1'
model = 'semseg'
data_mode = 'standard'
drr_data_path = data_path + patient_name + '/models/'+ model + '/' + data_mode +'/dataset/'
# Directory to save logs and trained model
LOG_DIR =  data_path + patient_name + '/models/' + model + '/' + data_mode +'/logs/'


imgs = dtn.load_data(drr_data_path + 'train_imgs.hdf5', 'train_imgs')
labs = dtn.load_data(drr_data_path + 'train_labs.hdf5', 'train_labs')
test_imgs = dtn.load_data(drr_data_path + 'test_imgs.hdf5', 'test_imgs')
test_labs = dtn.load_data(drr_data_path + 'test_labs.hdf5', 'test_labs')

**3. Preprocess loaded datasets**

In [ ]:
for i in range(labs.shape[0]):
    for j in range(labs.shape[1]):
        labs[i, j] = fit_ellipse(labs[i, j])
for i in range(test_labs.shape[0]):
    for j in range(test_labs.shape[1]):
        test_labs[i, j] = fit_ellipse(test_labs[i, j])
        
        
imgs = (imgs+1)/2.
test_imgs = (test_imgs+1)/2.

# TRAIN MODEL
s = test_imgs.shape
imgs = np.reshape(imgs, (-1, s[2], s[3]))[:, :, 64:-64]
labs = np.reshape(labs, (-1, s[2], s[3]))[:, :, 64:-64]
test_imgs = np.reshape(test_imgs, (-1, s[2], s[3]))[:, :, 64:-64]
test_labs = np.reshape(test_labs, (-1, s[2], s[3]))[:, :, 64:-64]

**4. Define training model**

In [ ]:
model = modellib.SiamMask(mode="training", config=config,
                                      model_dir=LOG_DIR)

**5. Train model**

In [ ]:
def lr_scheduler(epoch, lr):
    if epoch > 0 and epoch<=10 and lr < 1e-2:
        lr = lr + lr/epoch
    elif epoch > 10 and lr > 1e-5:
        k = 0.05
        lr = lr*np.exp(-k*epoch)
    return lr

lrs = LearningRateScheduler(lr_scheduler)  

model.train(imgs, labs, test_imgs, test_labs,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                layers='all',
                custom_callbacks = [lrs])